In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf
import json
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # 不使用GPU

In [4]:
import import_ipynb
from VGGModel import vgg

In [5]:
data_root = os.path.abspath(os.path.join(os.getcwd(),"../../datasets"))
image_path = data_root+"/flower_data/"
train_dir = image_path+'train'
validation_dir = image_path+'val'

In [6]:
im_height=224
im_width =224
batch_size=32
epochs = 10

In [18]:
# 论文里有去均值的预处理，这里没有是因为从头开始训练的不是迁移学习
# ImageNet RGB三通道均值是[123.68,116.78,103.94]
train_image_generator = ImageDataGenerator(rescale=1./255.,horizontal_flip=True)
validation_image_generator = ImageDataGenerator(rescale=1./255.)

train_data_gen = train_image_generator.flow_from_directory(directory=train_dir,
                                                          batch_size=batch_size,
                                                          shuffle=True,
                                                          target_size=(im_height,im_width),
                                                          class_mode = 'categorical')
total_train=train_data_gen.n

Found 3306 images belonging to 5 classes.


In [11]:
class_indices = train_data_gen.class_indices
class_indices

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}

In [12]:
inverse_dict = dict((val,key) for key,val in class_indices.items())

json_str = json.dumps(inverse_dict,indent=4)
with open('class_indices.json','w') as json_file:
    json_file.write(json_str)

In [13]:
val_data_gen = train_image_generator.flow_from_directory(directory=validation_dir,
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        target_size=(im_height,im_width),
                                                        class_mode='categorical')
total_val=val_data_gen.n

Found 364 images belonging to 5 classes.


In [14]:
model = vgg('vgg16',224,224,5)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
feature (Sequential)         (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dropout (Dropout)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 2048)              51382272  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              419635

In [16]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
            metrics=['accuracy'])

In [17]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath='./save_weights/myVGG_{epoch}.h5',
                                               save_best_only=True,
                                               save_weights_only=True,
                                               monitor='val_loss')]

In [ ]:
history = model.fit(x=train_data_gen,
                   steps_per_epoch=total_train//batch_size,
                   epochs=epochs,
                   validation_data=val_data_gen)